convert notebook to documentation?


In [5]:
var importer = require('../Core');
var getRpcFromSpec = importer.import('get rpc from spec');
var getPermissions = importer.import('rpc permissions');
var groupPermissions = importer.import('test rpc permissions');
var addAST = importer.import('get all ast properties')

function getRpcFromCells() {
    var functions = groupPermissions(['Function', 'Available']);
    var resources = Object.keys(functions).reduce((obj, f) => {
        var cell = addAST(importer.interpret(f));
        var notebook = cell.notebook.replace(/\.ipynb|\s+/ig, '').toLowerCase();
        if(typeof obj[notebook] === 'undefined') obj[notebook] = {methods: {}};
        obj[notebook].methods[cell.params[0]] = {
            description: cell.markdown,
            scopes: functions[f],
            id: `jupyter_ops.${notebook}.${cell.params[0]}`,
            httpMethod: 'GET',
            path: `?function=${cell.questions[0]}`,
            parameters: cell.params.slice(1).reduce((o, p) => (o[p] = {
                type: 'string',
                location: 'query',
                required: true,
                description: ''
            }, o), {})
        };
        return obj;
    }, {});
    return {
        title: 'Jupyter Ops',
        name: 'jupyter_ops',
        description: 'Automation notebooks in Jupyter',
        protocol: 'rest',
        baseUrl: process.env.FUNCTIONS_URL,
        resources: resources,
        auth: [],
        schemas: [],
        parameters: {}
    }
}

module.exports = getRpcFromCells;


[Function: getRpcFromCells]

test convert notebook documentation?


In [12]:
var importer = require('../Core');
var getRpcFromCells = importer.import('convert notebook to documentation');
var getRpcFromSpec = importer.import('get rpc from spec');
var getEnvironment = importer.import('get environment');
getEnvironment('STUDY_LOCAL')

if(typeof $$ !== 'undefined') {
    $$.async()
    var juypter_ops = getRpcFromSpec(getRpcFromCells());
    juypter_ops.studysauce.copyStudy({email: 'megamindbrian@gmail.com'})
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


reading notebook /Users/briancullinan/jupyter_ops/Analytics/documentation.ipynb
compiling /Users/briancullinan/jupyter_ops/Analytics/documentation.ipynb[0] aka convert notebook to documentation
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[8] aka get rpc from spec
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[2] aka rpc permissions
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[1] aka get cell rpc groups
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb[0] aka rpc groups
reading notebook /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipynb
compiling /Users/briancullinan/jupyter_ops/Cloud Services/rpc.ipyn

Error: Request failed with status code 500

convert google discovery to documentation?

https://developers.google.com/apis-explorer/#s/discovery/v1/discovery.apis.getRest?api=discovery&version=v1&_h=2&

https://www.googleapis.com/discovery/v1/apis/discovery/v1/rest

https://developers.google.com/apis-explorer/#p/discovery/v1/discovery.apis.list?_h=1&




discover well-known apis?

TODO: figure out how OpenAPI, Swagger RAML do this, Google's is obvious

https://tools.ietf.org/html/rfc5785
